In [ ]:
## 모듈 로딩 
import torch                                            ## 텐서 및 기본 함수들 모듈
import torch.nn as nn                                   ## 인공신경망 관련 모듈
import torch.nn.functional as F                         ## 인공신경망 관련 함수들 모듈
import torch.optim as optim                             ## 인공신경망 관련 최적화 모듈
from torch.optim.lr_scheduler import ReduceLROnPlateau  ## 학습률 조정 

from torchinfo import summary                           ## 모델 정보 및 구조 확인 모듈
from torchmetrics.classification import *               ## 모델 성능 지표 관련 모듈

from torchvision.datasets import ImageFolder            ## 이미지용 데이터셋 생성 모듈
from torch.utils.data import DataLoader                 ## 데이터 셋 관련 모듈
from torch.utils.data import Subset, random_split       
from torchvision.transforms import transforms           ## 이미지 전처리 및 증강 모듈

import matplotlib.pyplot as plt                         ## 이미지 시각화 

from utils2 import * 
import os


In [3]:
# 데이터 준비
TRAIN_ROOT = "./data/train/"
VAL_ROOT ='./data/valid/'
TEST_ROOT = './data/test/'

[2-1] 이미지 데이터 전처리용 인스턴스 생성

In [4]:
# 이미지 전처리 및 변경: 흑백, tensor화+정규화

train_transform = transforms.Compose(
    [
        transforms.Resize((48, 48)),  # 모든 이미지 크기를 48x48로 통일
        transforms.Grayscale(num_output_channels=1),  # Grayscale로 변환
        transforms.ToTensor(),                        # 텐서로 변환
        transforms.Normalize((0.5,), (0.5,))          # 채널 1개 -> 평균, 표준편차도 1개씩
    ]
)

test_transform = transforms.Compose(
    [
        transforms.Resize((48, 48)),  # 모든 이미지 크기를 48x48로 통일
        transforms.Grayscale(num_output_channels=1),  # Grayscale로 변환
        transforms.ToTensor(),                        # 텐서로 변환
        transforms.Normalize((0.5,), (0.5,))          # 채널 1개 -> 평균, 표준편차도 1개씩
    ]
)

valid_transform = transforms.Compose(
    [
        transforms.Resize((48, 48)),  # 모든 이미지 크기를 48x48로 통일
        transforms.Grayscale(num_output_channels=1),  # Grayscale로 변환
        transforms.ToTensor(),                        # 텐서로 변환
        transforms.Normalize((0.5,), (0.5,))          # 채널 1개 -> 평균, 표준편차도 1개씩
    ]
)



In [5]:
# 이미지 데이터 로딩 
trainDS = ImageFolder(root=TRAIN_ROOT, 
                    transform=train_transform)
testDS = ImageFolder(root=TEST_ROOT, 
                    transform=test_transform)
validDS = ImageFolder(root=VAL_ROOT, 
                    transform=valid_transform)


In [6]:
## - 클래스 변환 데이터 
TRAIN_IDX_TO_CLASS = {v:k for k, v in trainDS.class_to_idx.items()}
print(f'TRAIN_IDX_TO_CLASS => {TRAIN_IDX_TO_CLASS}')

TEST_IDX_TO_CLASS = {v:k for k, v in testDS.class_to_idx.items()}
print(f'TEST_IDX_TO_CLASS => {TEST_IDX_TO_CLASS}')

VAL_IDX_TO_CLASS = {v:k for k, v in validDS.class_to_idx.items()}
print(f'VAL_IDX_TO_CLASS => {VAL_IDX_TO_CLASS}')

TRAIN_IDX_TO_CLASS => {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}
TEST_IDX_TO_CLASS => {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}
VAL_IDX_TO_CLASS => {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}


In [51]:
## - 데이터 확인 (train)
print(f'trainDataset 개수 : {len(trainDS.targets)}개')
print(f'trainDataset 분류 : {trainDS.class_to_idx}')
print(f'- angry     개수 : {trainDS.targets.count(0)}개, {(trainDS.targets.count(0)/len(trainDS.targets))*100}%')
print(f'- disgust   개수 : {trainDS.targets.count(1)}개, {(trainDS.targets.count(1)/len(trainDS.targets))*100}%')
print(f'- fear      개수 : {trainDS.targets.count(2)}개, {(trainDS.targets.count(2)/len(trainDS.targets))*100}%')
print(f'- happy     개수 : {trainDS.targets.count(3)}개, {(trainDS.targets.count(3)/len(trainDS.targets))*100}%')
print(f'- neutral   개수 : {trainDS.targets.count(4)}개, {(trainDS.targets.count(4)/len(trainDS.targets))*100}%')
print(f'- sad       개수 : {trainDS.targets.count(5)}개, {(trainDS.targets.count(5)/len(trainDS.targets))*100}%')
print(f'- surprise  개수 : {trainDS.targets.count(6)}개, {(trainDS.targets.count(6)/len(trainDS.targets))*100}%')

trainDataset 개수 : 59586개
trainDataset 분류 : {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
- angry     개수 : 5355개, 8.987010371563791%
- disgust   개수 : 3231개, 5.422414661162018%
- fear      개수 : 30000개, 50.34739703957305%
- happy     개수 : 5000개, 8.391232839928843%
- neutral   개수 : 5000개, 8.391232839928843%
- sad       개수 : 5000개, 8.391232839928843%
- surprise  개수 : 6000개, 10.069479407914612%


In [7]:
# fear는 1로, 나머지는 모두 0으로 변경하기
def relabel_dataset(dataset):
    for i, (path, label) in enumerate(dataset.samples):
        if label == 2:  # 'fear' 라벨 번호가 2로 지정된 경우
            new_label = 1  # fear -> 1
        else:
            new_label = 0  # 나머지 모두 -> 0
        dataset.samples[i] = (path, new_label)

# 모든 데이터셋에 대해 라벨 변경하기
relabel_dataset(trainDS)
relabel_dataset(testDS)
relabel_dataset(validDS)

# 라벨이 제대로 변경되었는지 확인하기
train_labels = [label for _, label in trainDS.samples]
test_labels = [label for _, label in testDS.samples]
valid_labels = [label for _, label in validDS.samples]

print(f"Train Dataset - 0 (Others): {train_labels.count(0)}, 1 (Fear): {train_labels.count(1)}")
print(f"Test Dataset - 0 (Others): {test_labels.count(0)}, 1 (Fear): {test_labels.count(1)}")
print(f"Valid Dataset - 0 (Others): {valid_labels.count(0)}, 1 (Fear): {valid_labels.count(1)}")


Train Dataset - 0 (Others): 29586, 1 (Fear): 30000
Test Dataset - 0 (Others): 16665, 1 (Fear): 691
Valid Dataset - 0 (Others): 16697, 1 (Fear): 659


[2] 데이터 로딩 및 데이터셋 준비 <hr>

[3] 모델 정의 및 설계

In [8]:
class FEARDNN(nn.Module):
    def __init__(self, isDebug=False):
        super(FEARDNN, self).__init__()
        
        # 입력층: 이미지 크기가 48x48
        self.in_layer   = nn.Flatten()
        
        # 은닉층
        self.hd_layer1  = nn.Linear(48 * 48, 512) 
        self.drop_layer = nn.Dropout(0.25)
        self.hd_layer2  = nn.Linear(512, 256)
        self.hd_layer3  = nn.Linear(256, 130)
        
        # 출력층: 이진 분류이므로 출력 노드를 1개로 설정
        self.out_layer  = nn.Linear(130, 1)  
        
        # 디버그 모드 설정
        self.isDebug    = isDebug

    ## 순방향 학습 진행 메서드 
    def forward(self, data):
        ## 3D (BS, H, W) ==> 2D (BS, H*W)
        if self.isDebug: print(f'data shape : {data.shape}') # True 일때만 출력 (디버깅 용)
        
        out = self.in_layer(data)
        if self.isDebug: print(f'out shape : {out.shape}')

        out = F.relu(self.hd_layer1(out))
        out = self.drop_layer(out)

        out = F.relu(self.hd_layer2(out))
        out = self.drop_layer(out)

        out = F.relu(self.hd_layer3(out))
        out = self.out_layer(out)  # Sigmoid 제거 (BCEWithLogitsLoss가 Sigmoid 포함)
        
        if self.isDebug: print(f'out shape : {out.shape}')
        
        return out


In [24]:
class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=0.25, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        
        if self.reduction == 'mean':
            return torch.mean(F_loss)
        elif self.reduction == 'sum':
            return torch.sum(F_loss)
        else:
            return F_loss

[4] 학습 준비

In [21]:
from math import ceil

# 학습 설정
EPOCHS      =  50
BATCH_SIZE  = 1000

T_ITERATION = ceil(len(trainDS) / BATCH_SIZE)
V_ITERATION = ceil(len(testDS) / BATCH_SIZE)

print(f'T_ITERATION (Train): {T_ITERATION}')
print(f'V_ITERATION (Test): {V_ITERATION}')


# 최적화 설정
LR          = 0.001  # 학습률 (Learning Rate)
PAT_CNT     = 10     # 조기 종료 기준 (성능 개선 없을 때 참을 수 있는 횟수)
CLASSES     = len(trainDS.classes)  # 클래스 개수 확인

# 학습 장치 설정 (GPU 또는 CPU)
DEVICE      = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f'DEVICE => {DEVICE}')
print(f'T_ITERATION (Train): {T_ITERATION}')
print(f'V_ITERATION (Test): {V_ITERATION}')

T_ITERATION (Train): 60
V_ITERATION (Test): 18
DEVICE => cpu
T_ITERATION (Train): 60
V_ITERATION (Test): 18


In [25]:
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy

## 인스턴스 생성
GEN  = torch.Generator().manual_seed(42)
TRAINDL     = DataLoader(trainDS, batch_size=BATCH_SIZE, shuffle=True, generator=GEN)
TESTDL      = DataLoader(testDS,  batch_size=BATCH_SIZE, shuffle=True)
VALIDDL     = DataLoader(validDS,  batch_size=BATCH_SIZE, shuffle=True)


# 모델과 최적화 관련
MODEL       = FEARDNN().to(DEVICE)
OPTIMIZER   = optim.Adam(MODEL.parameters(), lr=LR)
SCHEDULER   = ReduceLROnPlateau(OPTIMIZER, mode='min', patience=PAT_CNT) 

# 손실함수, 모델성능 평가 관련 함수 인스턴스
LOSS_FN     = FocalLoss(alpha=0.25, gamma=2).to(DEVICE)
SCORE_FN    = BinaryF1Score().to(DEVICE)
ACC_FN      = BinaryAccuracy(threshold=0.5).to(DEVICE)


In [18]:
def evaluate(model, testDL, loss_fn, score_fn, acc_fn, n_iter):
    model.eval()
    T_LOSS, T_ACC, T_SCORE = 0, 0, 0

    with torch.no_grad():
        for feature, target in testDL:
            feature, target = feature.to(DEVICE), target.to(DEVICE).float().unsqueeze(1)

            # 모델 예측 (로짓 값 출력)
            pre_y = model(feature)

            # 손실 계산 (BCEWithLogitsLoss 사용)
            loss = loss_fn(pre_y, target)

            # 모델 출력값을 Sigmoid 함수로 확률로 변환
            prob = torch.sigmoid(pre_y)

            # 예측 값(0 또는 1)으로 변환 (명시적으로 threshold=0.5 적용)
            pred = (prob > 0.5).float()

            # 정확도와 F1 Score 계산 (acc_fn 사용)
            acc = acc_fn(prob, target.int())  # 확률 값 그대로 사용하여 BinaryAccuracy 계산
            score = score_fn(pred, target.int())
            
            # 디버깅을 위한 출력
            # print(f"Batch Accuracy (acc_fn): {acc:.5f}, Batch F1: {score:.5f}")

            T_LOSS += loss.item()
            T_SCORE += score.item()
            T_ACC  += acc.item()

    return T_LOSS / n_iter, T_SCORE / n_iter, T_ACC / n_iter


In [19]:
def training(model, trainDL, optimizer, loss_fn, score_fn, acc_fn, n_iter):
    model.train()
    E_LOSS, E_ACC, E_SCORE = 0, 0, 0

    for feature, target in trainDL:
        feature, target = feature.to(DEVICE), target.to(DEVICE).float().unsqueeze(1)

        # 가중치 기울기 초기화
        optimizer.zero_grad()

        # 예측값 계산 (로짓 값)
        pre_y = model(feature)

        # 손실 계산
        loss = loss_fn(pre_y, target)

        # 모델 출력값을 Sigmoid 함수로 확률로 변환
        prob = torch.sigmoid(pre_y)

        # 예측 값(0 또는 1)으로 변환
        pred = (prob > 0.5).float()

        # 정확도와 F1 Score 계산 (acc_fn 사용)
        acc = acc_fn(prob, target.int())
        score = score_fn(pred, target.int())
        
        # 디버깅을 위한 출력
        # print(f"Batch Accuracy (acc_fn): {acc:.5f}, Batch F1: {score:.5f}")

        # 역전파 진행
        loss.backward()
        optimizer.step()

        E_LOSS += loss.item()
        E_SCORE += score.item()
        E_ACC  += acc.item()

    return E_LOSS / n_iter, E_SCORE / n_iter, E_ACC / n_iter


In [ ]:
# ## 학습 관련 함수
# ## --------------------------------------------------------------
# ## - 학습 함수 : 학습 데이터셋 사용하는 함수 
# ##              W,b 업데이트 진행
# ## --------------------------------------------------------------
# def training(model, trainDL, optimizer, loss_fn, score_fn, acc_fn, n_iter):
#     # 학습 모드 설정
#     model.train()

#     E_LOSS, E_ACC, E_SCORE = 0, 0, 0
#     for feature, target in trainDL:
#         # 배치크기만큼 feature, target 로딩
#         feature, target = feature.to(DEVICE), target.to(DEVICE).float().unsqueeze(1)  # 타겟을 (N, 1) 형태로 변환
        
#         # 가중치 기울기 0 초기화
#         optimizer.zero_grad()

#         # 학습 진행
#         pre_y = model(feature)
        
#         # 손실 계산 (BCEWithLogitsLoss 사용 시 Sigmoid 필요 없음)
#         loss = loss_fn(pre_y, target)
        
#         # 예측값을 Sigmoid로 변환하여 확률로 만들기
#         pred = torch.sigmoid(pre_y) > 0.5  # 확률을 0.5 기준으로 이진 분류
        
#         # 정확도와 F1 Score 계산
#         score = score_fn(pred, target.int())
#         acc = acc_fn(pred, target.int())
        
#         # 역전파 진행
#         loss.backward()
        
#         # 가중치/절편 업데이트
#         optimizer.step()

#         E_LOSS += loss.item()
#         E_SCORE += score.item()
#         E_ACC  += acc.item()

#     return E_LOSS / n_iter, E_SCORE / n_iter, E_ACC / n_iter


학습 진행: 모델 또는 가중치 저장 + 조기종료

In [14]:
## 모델 경로 설정
MODEL_DIR  = './models/'
MODEL_FILE = 'FearModel.pt'

In [26]:

# 학습 기록 저장
HIST = {'Train': [[], []], 'Valid': [[], []]}  

# 모델저장을 위한 기준값 저장 변수
BEST_ACC = 0

# 조기 종료 위한 기준값 저장 변수
EARLY_STOP = 3  # patience count
STOP_CNT = 0    # Stop counter

# 에포크 단위 학습/검증 진행
for epoch in range(EPOCHS):
    # 모델 학습하기
    trainLoss, trainF1, trainAcc = training(MODEL, TRAINDL, OPTIMIZER, LOSS_FN, SCORE_FN, ACC_FN, T_ITERATION)
    
    # 모델 평가하기
    validLoss, validF1, validAcc = evaluate(MODEL, VALIDDL, LOSS_FN, SCORE_FN, ACC_FN, V_ITERATION)

    ## 모델 층별 가중치+바이어스 저장
    if BEST_ACC < validAcc:   # Best Accuracy 기준으로 모델 저장
        model_path = f'{MODEL_DIR}fear_weights_epoch{epoch}_{validAcc:.3f}.pt'
        
        # 모델 저장
        torch.save(MODEL.state_dict(), f'{MODEL_DIR}fear_weights_epoch{epoch}_{validAcc:.3f}.pt')
        BEST_ACC = validAcc
        STOP_CNT = 0  # 모델이 개선되면, 조기 종료 카운터 초기화
        
        #  모델 저장 메시지 출력
        print(f'\n 모델 저장됨: {model_path} (Valid Accuracy: {validAcc:.3f})')
    else:
        STOP_CNT += 1

    # 학습 상태 저장
    HIST['Train'][0].append(trainLoss) 
    HIST['Train'][1].append(trainAcc) 
    HIST['Valid'][0].append(validLoss) 
    HIST['Valid'][1].append(validAcc) 

    # 학습 상태 시각화
    print(f'\nEPOCH[{epoch+1}/{EPOCHS}]----------------')
    print(f'- TRAIN_LOSS {trainLoss:.5f}  F1 {trainF1:.5f}  ACC {trainAcc:.5f}')
    print(f'- VALID_LOSS {validLoss:.5f}  F1 {validF1:.5f}  ACC {validAcc:.5f}')
    
    # 학습률 조정 (Scheduler 사용) - 조기 종료 체크
    SCHEDULER.step(validLoss) 
    
    # 조기 종료 체크
    if SCHEDULER.num_bad_epochs >= SCHEDULER.patience: # 카운팅 # patience 10번 -> 10번 에포크 중에 성능 개선 안된 거에 대한 기준점ㅇ ㅣ필요 
         # 10번을 몇번 참을건지 정해야 함
        EARLY_STOP -=1
    if not EARLY_STOP:
        print(f'{epoch}EPOCHS: 성능 개선이 없어 조기종료') # 다만 여전히 모델이 남아 있음 -> 랜덤 웨이트를 초기부터 시작하는게 아니라 받아서 그다음부터 돌아가면 됨 
    
  


 모델 저장됨: ./models/fear_weights_epoch0_0.950.pt (Valid Accuracy: 0.950)

EPOCH[1/50]----------------
- TRAIN_LOSS 0.01951  F1 0.87496  ACC 0.87642
- VALID_LOSS 0.01661  F1 0.00975  ACC 0.94967

 모델 저장됨: ./models/fear_weights_epoch1_0.959.pt (Valid Accuracy: 0.959)

EPOCH[2/50]----------------
- TRAIN_LOSS 0.01525  F1 0.90601  ACC 0.91272
- VALID_LOSS 0.01505  F1 0.00851  ACC 0.95888

 모델 저장됨: ./models/fear_weights_epoch2_0.959.pt (Valid Accuracy: 0.959)

EPOCH[3/50]----------------
- TRAIN_LOSS 0.01471  F1 0.90763  ACC 0.91436
- VALID_LOSS 0.01514  F1 0.01295  ACC 0.95899

EPOCH[4/50]----------------
- TRAIN_LOSS 0.01476  F1 0.90589  ACC 0.91279
- VALID_LOSS 0.01434  F1 0.01056  ACC 0.95825

EPOCH[5/50]----------------
- TRAIN_LOSS 0.01409  F1 0.90989  ACC 0.91630
- VALID_LOSS 0.01419  F1 0.01099  ACC 0.95700

EPOCH[6/50]----------------
- TRAIN_LOSS 0.01361  F1 0.91227  ACC 0.91818
- VALID_LOSS 0.01437  F1 0.04561  ACC 0.95870

EPOCH[7/50]----------------
- TRAIN_LOSS 0.01388  F1 0.91

모델

In [51]:
%pip install streamlit

  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached jsonschema_specifications-2024.10.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   ------------------------------------ --- 8.9/9.8 MB 46.3 MB/s eta 0:00:01
   ---------------------------------------- 9.8/9.8 MB 40.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   --------------------------------------- 731.2/731.2 kB 31.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.5 MB ? eta -:--:--
   ------------ --------------------------- 8.1/25.5 MB 41.8 MB/s eta 0:00:01
   ------------------------- -------------- 16.5/25.5 MB 39.9 MB/s eta 0:00:01
   -------------------------------------- - 24.4/25.5 MB 40.6 MB/s eta 0:00:01
   --------------------------------

In [ ]:
#예측함수

def predict_image(model, image_path):
    from torchvision.transforms import transforms  
    import torch   
    
    # 이미지 전처리
    transform = transforms.Compose([
        transforms.Resize((48, 48)),
        transforms.Grayscale(num_output_channels=1),  # 흑백 이미지로 변환
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5]),  # 정규화
    ])
    
    img = Image.open(image_path)
    img = transform(img).unsqueeze(0)  # 배치 차원 추가
    
    # 예측
    with torch.no_grad():
        output = model(img)
    prediction = torch.sigmoid(output).item()  # sigm...



In [88]:

# 예측 함수 정의
def predict_image(model, image_path):
    # 이미지 전처리
    transform = transforms.Compose([
        transforms.Resize((48, 48)),
        transforms.Grayscale(num_output_channels=1),  # 흑백 이미지로 변환
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5]),  # 정규화
    ])
    
    # 이미지 로드 및 전처리
    img = Image.open(image_path).convert('RGB')  # 이미지를 RGB로 변환 후 전처리
    img = transform(img).unsqueeze(0)  # 배치 차원 추가

    # 모델 예측
    model.eval()  # 평가 모드로 설정
    with torch.no_grad():
        output = model(img)
    
    # 모델 출력 확인 (디버깅용)
    # print(f"모델 출력값 (로짓 값): {output.item()}")  # Sigmoid 적용 전 값

    # Sigmoid로 확률 값으로 변환
    probability = torch.sigmoid(output).squeeze().item()  # .squeeze()로 크기 조정 후 .item() 사용
    
    # 결과 해석
    if probability > 0.5:
        result = "Not Fear"
        confidence = probability * 100
    else:
        result = "Fear"
        confidence = (1 - probability) * 100

    return result, confidence

In [94]:
import torch

# 모델 인스턴스 생성 (정확히 동일한 구조여야 함)
model = FEARDNN()  # FEARDNN 클래스는 모델 학습 시 사용한 동일한 모델이어야 함

# 모델 가중치 로드하기
MODEL_PATH = './models/fear_weights_epoch2_0.959.pt'
model.load_state_dict(torch.load(MODEL_PATH, map_location=torch.device('cpu')))

# 모델을 평가 모드로 설정
model.eval()

C:\Users\KDP-28-\AppData\Local\Temp\ipykernel_14016\4165521011.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH, map_location=

FEARDNN(
  (in_layer): Flatten(start_dim=1, end_dim=-1)
  (hd_layer1): Linear(in_features=2304, out_features=512, bias=True)
  (drop_layer): Dropout(p=0.25, inplace=False)
  (hd_layer2): Linear(in_features=512, out_features=256, bias=True)
  (hd_layer3): Linear(in_features=256, out_features=130, bias=True)
  (out_layer): Linear(in_features=130, out_features=1, bias=True)
)

In [93]:
# 예측할 이미지 파일 경로 설정 (확인 필요)
image_path ='./data/test/happy/29026Exp3distressed_actor_378.jpg'
# image_path = './data/image2.png'
# 예측 실행
result, confidence = predict_image(model, image_path)
print(f"예측 결과: {result} (확률: {confidence:.2f}%)")


예측 결과: Fear (확률: 66.07%)
